In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score, precision_score
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
test = False

In [3]:
# load dataset
df = pd.read_csv('../data/dataset_clean.csv')
df.head()

,order_id,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,category_1_count,category_2_count,category_3_count,category_4_count,category_5_count,category_6_count,category_7_count,category_8_count,category_9_count,category_10_count,category_11_count,category_12_count,category_13_count,category_14_count,category_15_count,category_16_count,category_17_count,category_18_count,category_19_count,category_20_count,category_21_count,order_dow,order_hour_of_day
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,4,10
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,9
2,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5,17
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1,9
4,5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,3.0,3.0,0.0,4.0,1.0,0.0,6,16


In [4]:
if test:
    _, df, _, _ = train_test_split(df, df['category_1'],
                                   test_size=0.05,
                                   random_state=0,
                                   stratify=df[['order_dow', 'order_hour_of_day']])

In [5]:
# dow one hot
enc = OneHotEncoder()
df[[f"order_dow_{x}" for x in range(7)]] = enc.fit_transform(df.loc[:, 'order_dow'].values.reshape((-1, 1))).toarray()

# hod one hot
enc = OneHotEncoder()
df[[f"order_hod_{x}" for x in range(24)]] = enc.fit_transform(df.loc[:, 'order_hour_of_day'].values.reshape((-1, 1))).toarray()
df.head()

,order_id,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,category_1_count,category_2_count,category_3_count,category_4_count,category_5_count,category_6_count,category_7_count,category_8_count,category_9_count,category_10_count,category_11_count,category_12_count,category_13_count,category_14_count,category_15_count,category_16_count,category_17_count,category_18_count,category_19_count,category_20_count,category_21_count,order_dow,order_hour_of_day,order_dow_0,order_dow_1,order_dow_2,order_dow_3,order_dow_4,order_dow_5,order_dow_6,order_hod_0,order_hod_1,order_hod_2,order_hod_3,order_hod_4,order_hod_5,order_hod_6,order_hod_7,order_hod_8,order_hod_9,order_hod_10,order_hod_11,order_hod_12,order_hod_13,order_hod_14,order_hod_15,order_hod_16,order_hod_17,order_hod_18,order_hod_19,order_hod_20,order_hod_21,order_hod_22,order_hod_23
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,4,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5,17,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1,9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,3.0,3.0,0.0,4.0,1.0,0.0,6,16,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Helper functions

In [6]:
def partition_dataset(df,
                      label_col,
                      features=None,
                      partition_on=None,
                      test_size=0.2,
                      valid_size=0.2,
                      random_state=None,
                      three_way=False
                     ):
    if features is None:
        features = [col for col in df.columns if col != label_col]
    X = df[features]
    y = df[label_col].apply(int).values.reshape((-1, 1))
    if partition_on is not None:
        partition_on = df[partition_on]
        
    if three_way:
        rng = np.random.default_rng(random_state)
        seeds = rng.integers(10000, size=2)
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=seeds[0], 
                                                        stratify=partition_on
                                                       )
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                              test_size=test_size, 
                                                              random_state=seeds[1], 
                                                              stratify=partition_on
                                                             )
        return X_train, X_valid, X_test, y_train, y_valid, y_test
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=test_size, 
                                                            random_state=random_state, 
                                                            stratify=partition_on
                                                           )
        return X_train, X_test, y_train, y_test

In [7]:
# train test split stratified by label & dow
def train_xgb_cv_classifier(X, y,
                            test_size=0.2,
                            n_folds=5,
                            xgb_params={},
                            random_state=None
                           ):
    
    # handling xgb params and the defaults
    xgb_params_complete = {'n_estimators': 1000,
                           'learning_rate': 0.01,
                           'colsample_bytree': 0.8,
                           'subsample': 0.8}
    for key, val in xgb_params.items():
        xgb_params_complete[key] = val
    
    all_scores = []
    # CV fold
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for train_index, valid_index in skf.split(X, y):
        X_train, X_valid = X.iloc[train_index, :], X.iloc[valid_index, :]
        y_train, y_valid = y[train_index], y[valid_index]
        
        # xgb training
        model = XGBClassifier(**xgb_params_complete, 
                              use_label_encoder=False,
                             )
        model.fit(X_train, y_train)
        
        # prediction on validation set
        pred = model.predict(X_valid)
        pred = (pred > 0.5).astype(int)
        fold_scores = [accuracy_score(y_valid, pred),
                       precision_score(y_valid, pred),
                       recall_score(y_valid, pred),
                       f1_score(y_valid, pred)]
        all_scores.append(fold_scores)
        
    return np.array(all_scores).mean(axis=0)

## Estimating n_estimators

#### indicator var

In [8]:
labels = [f"category_{x}" for x in [4, 16, 7, 19, 1]]
col_dow = [f"order_dow_{x}" for x in range(7)]
col_hod = [f"order_hod_{x}" for x in range(24)]

In [9]:
pos_multipliers = np.arange(1, 3, 0.2)

for i, mult in enumerate(pos_multipliers):
    
    scores_all_labels = []

    for label in labels:

        # DATA SETUP

        features = ([f"category_{x}" for x in range(1, 22) if f"category_{x}" != label]
        #             + [f"category_{x}_count" for x in range(1, 22) if f"category_{x}_count" != label+'_count']
        #             + col_dow
        #             + col_hod
                   )

        # setup train and test set
        df_xgb = df[features + [label]]
        X_train_, X_test, y_train_, y_test = partition_dataset(df_xgb, 
                                                               label, 
                                                               features=features,
                                                               three_way=False,
                                                               partition_on=label,
                                                               random_state=0)

        # PARAMS SETUP

        xgb_params = {'learning_rate': 0.01,
                      'colsample_bytree': 0.8,
                      'subsample': 0.8,
                      'objective': 'binary:logistic',
                      'scale_pos_weight': len(y_train_[y_train_ == 0])/len(y_train_[y_train_ == 1]) * mult,
                      'eval_metric': 'logloss'
                     }


        # LOOP TO ESTIMATE N_ESITMATORS OVER DIFFERENT LABELS

        lengths = []
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        for train_index, valid_index in skf.split(X_train_, y_train_):
            X_train, X_valid = X_train_.iloc[train_index, :], X_train_.iloc[valid_index, :]
            y_train, y_valid = y_train_[train_index], y_train_[valid_index]

            dtrain = xgb.DMatrix(X_train, label=y_train)
            dvalid = xgb.DMatrix(X_valid, label=y_valid)
            watchlist = [(dtrain,'train'), (dvalid,'valid')]

            model = xgb.train(xgb_params,
                              dtrain,
                              num_boost_round=5000,
                              evals=watchlist,
                              early_stopping_rounds=20,
                              verbose_eval=200)
            # get number of trees
            lengths.append(len(model.get_dump()))

        # n_estimator as average over folds
        n_estimators = int(np.array(lengths).mean())
        xgb_params['n_estimators'] = n_estimators


        # LOOP TO GET CV SCORES

        print(f"training {label}")
        scores_all_labels.append(
            train_xgb_cv_classifier(X_train, 
                                    y_train,
                                    xgb_params=xgb_params,
                                    random_state=1
                                   )
        )


    # final processing of scores    
    df_cv_scores = pd.DataFrame(np.stack(scores_all_labels),
                                columns=['accuracy', 'precision', 'recall', 'f1'],
                                index=labels)
    df_cv_scores.to_csv(f"../results/xgb_cv_v{i}.csv")

[0]	train-logloss:0.69227	valid-logloss:0.69224
[200]	train-logloss:0.61243	valid-logloss:0.61275
[400]	train-logloss:0.60491	valid-logloss:0.60517
[600]	train-logloss:0.60278	valid-logloss:0.60323
[800]	train-logloss:0.60172	valid-logloss:0.60229
[1000]	train-logloss:0.60064	valid-logloss:0.60127
[1200]	train-logloss:0.59928	valid-logloss:0.59996
[1400]	train-logloss:0.59831	valid-logloss:0.59915
[1600]	train-logloss:0.59758	valid-logloss:0.59845
[1800]	train-logloss:0.59688	valid-logloss:0.59791
[2000]	train-logloss:0.59643	valid-logloss:0.59753
[2200]	train-logloss:0.59609	valid-logloss:0.59718
[2400]	train-logloss:0.59578	valid-logloss:0.59692
[2550]	train-logloss:0.59540	valid-logloss:0.59672
[0]	train-logloss:0.69227	valid-logloss:0.69223
[200]	train-logloss:0.61269	valid-logloss:0.61260
[400]	train-logloss:0.60520	valid-logloss:0.60517
[600]	train-logloss:0.60333	valid-logloss:0.60346
[800]	train-logloss:0.60226	valid-logloss:0.60244
[1000]	train-logloss:0.60101	valid-logloss:0.

[1400]	train-logloss:0.65897	valid-logloss:0.65960
[1600]	train-logloss:0.65852	valid-logloss:0.65925
[1800]	train-logloss:0.65814	valid-logloss:0.65899
[2000]	train-logloss:0.65795	valid-logloss:0.65877
[2200]	train-logloss:0.65775	valid-logloss:0.65857
[2232]	train-logloss:0.65763	valid-logloss:0.65856
[0]	train-logloss:0.69288	valid-logloss:0.69280
[200]	train-logloss:0.66609	valid-logloss:0.66604
[400]	train-logloss:0.66278	valid-logloss:0.66291
[600]	train-logloss:0.66168	valid-logloss:0.66187
[800]	train-logloss:0.66088	valid-logloss:0.66113
[1000]	train-logloss:0.66023	valid-logloss:0.66046
[1200]	train-logloss:0.65942	valid-logloss:0.65989
[1400]	train-logloss:0.65894	valid-logloss:0.65944
[1600]	train-logloss:0.65853	valid-logloss:0.65912
[1800]	train-logloss:0.65818	valid-logloss:0.65880
[2000]	train-logloss:0.65797	valid-logloss:0.65856
[2200]	train-logloss:0.65776	valid-logloss:0.65836
[2317]	train-logloss:0.65759	valid-logloss:0.65829
training category_7
[0]	train-logloss:

KeyboardInterrupt: 

In [ ]:
df_xgb